In [1]:
import pandas as pd
from surprise import Dataset, Reader, SVD
from surprise.model_selection import train_test_split
from surprise import accuracy

In [2]:
ratings_file = '/kaggle/input/movielens-100k-dataset/ml-100k/u.data'
movies_file = '/kaggle/input/movielens-100k-dataset/ml-100k/u.item'

In [3]:
reader = Reader(line_format='user item rating timestamp', sep='\t')
data = Dataset.load_from_file(ratings_file, reader=reader)

In [4]:
movies = pd.read_csv(movies_file, sep='|', header=None, usecols=[0, 1], names=['movie_id', 'movie_title'], encoding='latin-1')


In [5]:
trainset, testset = train_test_split(data, test_size=0.25)

In [6]:

model = SVD()
model.fit(trainset)

In [7]:

predictions = model.test(testset)
print(f'RMSE: {accuracy.rmse(predictions)}')

RMSE: 0.9401
RMSE: 0.9400500588445746


In [13]:
def get_movie_title(movie_id):
    '''Helper function to get the movie title from movie_id.'''
    movie_title = movies[movies['movie_id'] == int(movie_id)]['movie_title']
    return movie_title.values[0] if not movie_title.empty else None

def recommend_movies(user_id, num_recommendations=5):
    '''Recommend movies for a user.'''
    
    movie_ids = movies['movie_id'].astype(str).tolist()
    
    
    predictions = [model.predict(user_id, movie_id) for movie_id in movie_ids]
    
    
    top_predictions = sorted(predictions, key=lambda x: x.est, reverse=True)[:num_recommendations]
    
    
    recommended_movies = [(pred.iid, get_movie_title(pred.iid)) for pred in top_predictions]
    return recommended_movies

In [14]:
user_id = str(1)  
recommendations = recommend_movies(user_id)
print(f"Recommendations for user {user_id}:")
for movie_id, title in recommendations:
    print(f"Movie ID: {movie_id}, Title: {title}")

Recommendations for user 1:
Movie ID: 64, Title: Shawshank Redemption, The (1994)
Movie ID: 223, Title: Sling Blade (1996)
Movie ID: 169, Title: Wrong Trousers, The (1993)
Movie ID: 100, Title: Fargo (1996)
Movie ID: 198, Title: Nikita (La Femme Nikita) (1990)
